In [1]:
import langchain

from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='Insert your key',
    model_name = "llama3-70b-8192"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

That's an easy one!

The first person to set foot on the Moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to walk on the Moon.


In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-38844?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

# Remember to provide an active job posting link in this section - cuz , this link be taken down by nike

Apply for Software Engineer I

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer IBeaverton, OregonBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does more than

In [7]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
    ###SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ###INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in a JSON format containing the
    following key: 'role','experience','skills' and 'description'.
    Only return the valid JSON
    ### VALID JSON (NO PREAMBLE):
    """

)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)
#This is an 'str' type data which we want in a JSON format

Here is the extracted job posting in JSON format:

```
{
  "role": "Software Engineer I",
  "experience": "2+ years of professional experience in application engineering",
  "skills": [
    "JavaScript (Node.js, React.js, Vue.js, etc.)",
    "React.js",
    "Redux",
    "AWS CloudFront",
    "AWS S3",
    "AWS Route 53",
    "AWS Certificate Manager",
    "AWS Lambda",
    "GitHub Co-pilot"
  ],
  "description": "Design and develop technical solutions that meet the needs of the business and aligns with Nike architectural and quality standards. Build mobile first Front-End applications in a manner visually and culturally consistent with Nike brand using React.js and Redux. Ensure SLAs (Service Level Agreements) for different interfaces are met and test them by executing performance tests using various loads. Provide production support on new and existing applications."
}
```


In [8]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer I',
 'experience': '2+ years of professional experience in application engineering',
 'skills': ['JavaScript (Node.js, React.js, Vue.js, etc.)',
  'React.js',
  'Redux',
  'AWS CloudFront',
  'AWS S3',
  'AWS Route 53',
  'AWS Certificate Manager',
  'AWS Lambda',
  'GitHub Co-pilot'],
 'description': 'Design and develop technical solutions that meet the needs of the business and aligns with Nike architectural and quality standards. Build mobile first Front-End applications in a manner visually and culturally consistent with Nike brand using React.js and Redux. Ensure SLAs (Service Level Agreements) for different interfaces are met and test them by executing performance tests using various loads. Provide production support on new and existing applications.'}

In [9]:
type(json_res)
#Type of this is 'dictionary' object - dict or JSON they are the same type

dict

In [10]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [11]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="my_portfolio")

if not collection.count():
   for _,row in df.iterrows():
     collection.add (documents=row["Techstack"],
                    metadatas={"links": row["Links"]},
                    ids=[str(uuid.uuid4())])

C:\Users\eshan\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:31<00:00, 2.64MiB/s]  


In [14]:
links = collection.query(query_texts=job['skills'],n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/react-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-back

{'role': 'Software Engineer I',
 'experience': '2+ years of professional experience in application engineering',
 'skills': ['JavaScript (Node.js, React.js, Vue.js, etc.)',
  'React.js',
  'Redux',
  'AWS CloudFront',
  'AWS S3',
  'AWS Route 53',
  'AWS Certificate Manager',
  'AWS Lambda',
  'GitHub Co-pilot'],
 'description': 'Design and develop technical solutions that meet the needs of the business and aligns with Nike architectural and quality standards. Build mobile first Front-End applications in a manner visually and culturally consistent with Nike brand using React.js and Redux. Ensure SLAs (Service Level Agreements) for different interfaces are met and test them by executing performance tests using various loads. Provide production support on new and existing applications.'}

In [13]:
job=json_res
job['skills']
#For each skill above we have the relevant links

['JavaScript (Node.js, React.js, Vue.js, etc.)',
 'React.js',
 'Redux',
 'AWS CloudFront',
 'AWS S3',
 'AWS Route 53',
 'AWS Certificate Manager',
 'AWS Lambda',
 'GitHub Co-pilot']

In [16]:
prompt_email = PromptTemplate.from_template(
    """
    ###JOB DESCRIPTION:
    {job_description}

    ###INSTRUCTION:
    You are Mohan, a business development executive at AtliQ.AtliQ is an AI and software consulting company dedicated to
    the seamless integration of business processes through automated tools.
    Over our experience, we have empowered numerous enterprises with tailored solutions ,
    process optimization, cost reduction, and hegightened overall eficiency.
    Your job is to write a cold email to the client regarding the job mentioned abovedescribing the capability
    in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase AtliQ's portfolio: {link_list}
    Remember you are Mohan , BDE at AtliQ.
    Do no provide a preamble.
    ### EMAIL (NO PREAMBLE):

    """
)

chain_email = prompt_email | llm
res = chain_email.invoke ({ "job_description": str(job), "link_list":links })
print(res.content)

Subject: Expert Software Engineering Solutions for Nike's Technical Needs

Dear Hiring Manager,

I came across the job description for a Software Engineer I role at Nike, and I'm excited to introduce AtliQ, a seasoned AI and software consulting company, as a potential partner to fulfill your technical requirements.

Our team of expert software engineers has a proven track record of designing and developing technical solutions that meet the highest standards of quality and performance. We specialize in building mobile-first Front-End applications using React.js and Redux, ensuring visually and culturally consistent experiences with your brand.

We've worked extensively with AWS CloudFront, AWS S3, AWS Route 53, AWS Certificate Manager, and AWS Lambda, ensuring seamless integration and optimal performance. Our expertise in GitHub Co-pilot enables us to streamline development workflows and improve collaboration.

To give you a glimpse into our capabilities, I'd like to share some relevant